## 星等转换为Flux单位

In [89]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from astropy.time import Time 
namelist = ['2003MA3','2010CO1','2010DG77','2010EX11','2010FC81','2010FH81','2010GS7','2010JN71','2010KX7','2010PW58','2010XP69','2019DD2']
for i in range(len(namelist)):
    name = namelist[i]
    rawdata = pd.read_csv(f'table_{name}.csv')
    data = rawdata.loc[:,['mjd','w1mpro','w2mpro','w3mpro','w4mpro']]
    data.insert(loc=len(data.columns),column='date',value='no')
    #data['mjd'] = 2400000.5+data['mjd']
    for i in tqdm(range(len(data))):  
        t = Time(data['mjd'][i],format='mjd',scale='utc')
        t.format = 'iso'
        data['date'][i] = t.value
    data.to_csv(f'{name}.txt',sep='\t',index=False)

  0%|          | 0/15 [00:00<?, ?it/s]/tmp/ipykernel_7561/627851880.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'][i] = t.value
100%|██████████| 15/15 [00:00<00:00, 2811.82it/s]


In [109]:
data[['date','mjd']]

,date,mjd
0,2010-03-03 10:56:44.189,55258.456067
1,2010-03-03 10:56:55.248,55258.456195
2,2010-03-03 12:32:05.251,55258.522283
3,2010-03-03 14:07:26.314,55258.588499
4,2010-03-03 15:42:36.317,55258.654587
5,2010-03-03 18:53:07.382,55258.786891
6,2010-03-03 20:28:28.445,55258.853107
7,2010-03-03 22:03:49.507,55258.919323
8,2010-06-05 07:18:14.976,55352.304340
9,2010-06-05 12:04:07.104,55352.502860


In [106]:

df = pd.read_table('mpc_final_sort.txt')
df = df.loc[:229]
for i in range(len(df)):
    if df['PHA'][i]=='Potentially':
        df['PHA'][i] = 'Yes'
    else:
        df['PHA'][i] = 'No'
df = df.sort_values(by=['PHA','time','Diam'],ascending=False,ignore_index=True)
df.to_excel('PHA.xlsx')

/tmp/ipykernel_7561/1341410745.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PHA'][i] = 'No'
/tmp/ipykernel_7561/1341410745.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PHA'][i] = 'Yes'
